In [2]:
from collections import defaultdict
from pathlib import Path

import evaluate
import numpy as np
from dotenv import load_dotenv
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.database import Database

import pandas as pd

In [3]:
def get_env_path() -> Path:
    return Path('..\\.env')


load_dotenv(get_env_path())

CONNECTION_STRING = f'mongodb://localhost:27017'
CLIENT = MongoClient(CONNECTION_STRING)
DB: Database = CLIENT['thesis']
# noinspection SpellCheckingInspection
TABLE_MITI: Collection = DB['miti']

In [4]:
_df_existing_evals = pd.read_csv('llama2_eval.csv', index_col=False)

In [5]:
_df_existing_evals

,Unnamed: 0,Model,bertscore,bertscore_f1_mean,bertscore_f1_std,bertscore_precision_mean,bertscore_recall_mean,bleu,bleurt,bleurt_f1_mean,...,frugalscore_f1_mean,frugalscore_f1_std,google_bleu,meteor,predicted_utterances_count,predicted_words_count_mean,predicted_words_count_std,reference_words_count_mean,reference_words_count_std,rouge
0,0,finetuning/miti_alexander_street/meta-llama-Ll...,"{'precision': [0.8521035313606262, 0.889962553...",0.857203,0.043770,0.858164,0.857271,"{'bleu': 0.04714233420857152, 'precisions': [0...","{'scores': [-1.5883477926254272, -0.5129691958...",-0.963470,...,0.006566,0.276434,{'google_bleu': 0.058391085206639076},{'meteor': 0.2036675683351478},6652,29.507817,33.912126,25.904991,29.444664,"{'rouge1': 0.2286281584299184, 'rouge2': 0.076..."
1,1,finetuning/alexander_street_small/meta-llama-L...,"{'precision': [0.8802219033241272, 0.872217297...",0.846896,0.029539,0.853084,0.842088,"{'bleu': 0.012781297138857287, 'precisions': [...","{'scores': [-1.4992144107818604, -1.6273263692...",-1.145407,...,-0.098164,0.157977,{'google_bleu': 0.03455417966787487},{'meteor': 0.1255303343638163},6652,24.511876,35.467151,25.904991,29.444664,"{'rouge1': 0.14487632150894864, 'rouge2': 0.02..."
2,2,finetuning/alexander_street_large/meta-llama-L...,"{'precision': [0.8678842782974243, 0.839466869...",0.848894,0.031134,0.858908,0.840558,"{'bleu': 0.01131199593102045, 'precisions': [0...","{'scores': [-1.2641234397888184, -1.3700337409...",-1.122640,...,-0.097693,0.169693,{'google_bleu': 0.03292305001123111},{'meteor': 0.11923888191444706},6652,22.558178,33.688205,25.904991,29.444664,"{'rouge1': 0.1385491822036144, 'rouge2': 0.020..."
3,3,meta-llama/Llama-2-7b-chat-hf,"{'precision': [0.8176425695419312, 0.863284587...",0.810307,0.022486,0.781226,0.842561,"{'bleu': 0.005177354701638891, 'precisions': [...","{'scores': [-0.9790810942649841, -0.7483015656...",-1.317601,...,-0.094603,0.121680,{'google_bleu': 0.019129019671542388},{'meteor': 0.1631574496909516},6652,173.048256,33.089070,25.904991,29.444664,"{'rouge1': 0.10993872775780206, 'rouge2': 0.01..."
4,4,finetuning/alexander_street_large/meta-llama-L...,"{'precision': [0.8413432836532593, 0.857688665...",0.846356,0.028436,0.853447,0.840669,"{'bleu': 0.01036091804679258, 'precisions': [0...","{'scores': [-1.0119128227233887, -1.6825077533...",-1.172437,...,-0.104471,0.151046,{'google_bleu': 0.03286954239250308},{'meteor': 0.11967210638136731},6652,23.107637,32.997601,25.904991,29.444664,"{'rouge1': 0.13922652976032834, 'rouge2': 0.02..."
5,5,finetuning/alexander_street_small/meta-llama-L...,"{'precision': [0.9089345932006836, 0.853552818...",0.846750,0.030293,0.852244,0.842509,"{'bleu': 0.012359712161719907, 'precisions': [...","{'scores': [-1.038384199142456, -0.95797485113...",-1.148234,...,-0.093074,0.160853,{'google_bleu': 0.03529539773505573},{'meteor': 0.12678145049351427},6652,24.351022,33.650652,25.904991,29.444664,"{'rouge1': 0.1467090228221501, 'rouge2': 0.023..."
6,6,finetuning/miti_alexander_street/meta-llama-Ll...,"{'precision': [0.8477878570556641, 0.875392675...",0.861267,0.047765,0.864269,0.859231,"{'bleu': 0.07003506053910136, 'precisions': [0...","{'scores': [-1.0559653043746948, -0.9023092389...",-0.898174,...,0.031425,0.310489,{'google_bleu': 0.07078532481310602},{'meteor': 0.21671404789969428},6652,24.929946,27.325215,25.904991,29.444664,"{'rouge1': 0.249585850179374, 'rouge2': 0.0966..."
7,7,finetuning/miti/meta-llama-Llama-2-13b-chat-hf-6,"{'precision': [0.8576045036315918, 0.864270389...",0.862704,0.050348,0.865171,0.861135,"{'bleu': 0.0607435130248533, 'precisions': [0....","{'scores': [-0.8965473175048828, -0.7514126896...",-0.858924,...,0.050345,0.323301,{'google_bleu': 0.06486506624350302},{'meteor': 0.22925758259801637},6652,25.274354,28.995102,25.904991,29.444664,"{'rouge1': 0.2642435855610533, 'rouge2': 0.105..."
8,8,finetuning/miti/meta-llama-Llama-2-7b-chat-hf-6,"{'precision': [0.8628348112106323, 0.868188798...",0.858311,0.045633,0.85

In [28]:
_df_existing_evals['bertscore'].dtype

dtype('O')

In [15]:
import json

In [23]:
_df_existing_evals['bertscore'].apply(lambda x: np.mean(x))

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U394318'), dtype('<U394318')) -> None

In [9]:
from scipy.stats import f_oneway
from statsmodels.sandbox.stats.multicomp import TukeyHSDResults
from statsmodels.stats.multicomp import pairwise_tukeyhsd


def calculate_anova(scores: pd.DataFrame, verbose: bool = False) -> TukeyHSDResults:
    models_name = scores['Model'].unique().tolist()

    func = lambda model_name: scores[scores['Model'] == model_name]['bertscore']['precision']

    # Perform One-Way ANOVA
    anova_result = f_oneway(*[func(model_name=model_name) for model_name in models_name])

    if verbose:
        print(f"ANOVA Result:\n{anova_result}\n")

    # If ANOVA is significant, perform post hoc test
    if anova_result.pvalue < 0.05:
        # Prepare data for Tukey's HSD test
        tukey_data = scores[['Model', 'Mean Value']]
        tukey_test = pairwise_tukeyhsd(endog=tukey_data['Mean Value'], groups=tukey_data['Model'], alpha=0.05)

        if verbose:
            print("Tukey's HSD Test Result:")
            print(tukey_test)

        return tukey_test

In [10]:
calculate_anova(_df_existing_evals, verbose=True)

KeyError: 'precision'

In [1]:
_df_existing_evals

NameError: name '_df_existing_evals' is not defined

In [5]:
_df = None
for _doc in TABLE_MITI.find({'llama2': {'$exists': True}}):
    _true_response = _doc['prompt']['true_response']
    _llama2 = _doc['llama2']
    _llama2['true_response'] = _true_response

    _df_tmp = pd.DataFrame.from_dict({k: [v] for k, v in _llama2.items()})
    if _df is None:
        _df = _df_tmp
    else:
        _df = pd.concat([_df, _df_tmp], ignore_index=True)

In [6]:
_df

,finetuning/miti_alexander_street/meta-llama-Llama-2-7b-chat-hf-6,finetuning/alexander_street_small/meta-llama-Llama-2-7b-chat-hf-6,finetuning/alexander_street_large/meta-llama-Llama-2-7b-chat-hf-6,meta-llama/Llama-2-7b-chat-hf,meta-llama/Llama-2-13b-chat-hf,finetuning/alexander_street_large/meta-llama-Llama-2-13b-chat-hf-6,finetuning/alexander_street_small/meta-llama-Llama-2-13b-chat-hf-6,finetuning/miti_alexander_street/meta-llama-Llama-2-13b-chat-hf-6,finetuning/miti/meta-llama-Llama-2-13b-chat-hf-6,finetuning/miti/meta-llama-Llama-2-7b-chat-hf-6,true_response
0,"okay, what kind of workouts do you usually do?","So, you're going to school and you're working ...",So what do you do when you're not going to sch...,"okay, got it. So, can you tell me a little bit...","I see, yeah okay and do you do you have any th...",And are you planning on going back to school a...,"Okay, so you're taking a break from school?","You work out, that's good! I'm curious because...",So you're a typical college student. So you sa...,"alright, well you mentioned that you like to w...","Well, they sounds like you're really, really h..."
1,"Okay. Well, you seem like you're feeling a lit...","Sure, sure.",I guess so.,"Okay, I see. So, it sounds like you're feeling...","I mean, it sounds like it sounds like you're f...",They love to have you home with them all the t...,"Well, they love you and they want to help you,...","yeah, so, it sounds like you are you're really...",You're feeling torn almost like you're pulling...,"Well, it sounds like you are a motivated perso...","So, it seems like your kind of conflicted abou..."
2,"Yeah, so it sounds like you're saying that you...","But, you know, they probably care about you, a...",How much time are you home?,It seems like you're feeling kind of frustrate...,"So, it sounds like you're feeling really confl...","Right. I mean, what's a little overly concerne...","Yeah, I mean, well, I'm sure it's just you kno...",Overly concerned?,"Well, it seems like your parents have a lot of...","Okay, so it seems like your family is really w...",Can you tell a little bit more about what you ...
3,got really exhausted.,Uh-huh.,"Yeah, well, I wonder if the, the symptoms you'...","I see, so it sounds like you were experiencing...","Okay, it sounds like it's not just been that y...","Okay, so, so that's what your parents were res...",Did you feel tired or you were feeling like ti...,"so, it just sound like you were kind of draggi...","So, you're really fatigued.",and you don't feel like that's related to stress.,"so, it sounds like you're really confused abou..."
4,You said that your family is kind of concerned...,But I guess my question is how are you feeling...,"Yeah, and you're a little concerned about your...","Okay, and you mentioned that your parents are ...",Can you tell me more about that? What do you m...,"Yeah, what do you think you were experiencing,...",Right.,I do know that your parents are definitely con...,So you don't have any idea why you get the sym...,"So, it seems like you're really confused about...",I know that you really discuss that working ou...
...,...,...,...,...,...,...,...,...,...,...,...
6647,yeah,oh yeah I mean I was on the phone with someone...,okay.,yah right yeah definitely yeah yeah definitely...,yeah\n user\nI mean my my my healthcare provid...,yeah but you're not ready to commit to that I ...,yeah sure yeah it's something that if you want...,absolutely,yeah,and so you could see yourself doing something ...,absolutely
6648,well it's important that you've identified tha...,okay and the idea is to not stop smoking right...,yeah I'd recommend you know look into that.,yeah yeah I see what you mean I think that the...,yeah i got it so you know you've been having d...,yeah I I'm really glad you asked because I thi...,yeah so well let's I mean one of the things th...,yeah well you've taken the first step Will by ...,yeah I guess what I'm hearing you say is that ...,so you've heard mixed messages that 

In [7]:
_bleu = evaluate.load("bleu")
_rouge = evaluate.load("rouge")
_meteor = evaluate.load("meteor")
_bertscore = evaluate.load("bertscore", device='cuda:0')
_frugalscore = evaluate.load("frugalscore", "moussaKam/frugalscore_medium_bert-base_mover-score")
_google_bleu = evaluate.load("google_bleu")
_bleurt = evaluate.load("bleurt", module_type="metric")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\stav3\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stav3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\stav3\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
C:\Users\stav3\anaconda3\envs\Thesis_GPTJ_Flask\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results wi


INFO:tensorflow:Reading checkpoint C:\Users\stav3\.cache\huggingface\metrics\bleurt\default\downloads\extracted\1dfb731fe2846298242021b3971e53cd2b22233cb0c4fdd87721b208630c396f\bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [8]:
_eval = defaultdict(defaultdict)

for _k in _df.columns:
    if _k == 'true_response':
        continue
    if _df_existing_evals['Model'].isin([_k]).any():
        continue

    print(f'Processing {_k} ...')

    _predictions = _df[_k].tolist()
    _references = _df['true_response'].tolist()
    _eval[_k]['predicted_utterances_count'] = len(_df)
    _eval[_k]['predicted_words_count_mean'] = np.mean(_df[_k].apply(lambda x: len(x.split())))
    _eval[_k]['predicted_words_count_std'] = np.std(_df[_k].apply(lambda x: len(x.split())))
    _eval[_k]['reference_words_count_mean'] = np.mean(_df['true_response'].apply(lambda x: len(x.split())))
    _eval[_k]['reference_words_count_std'] = np.std(_df['true_response'].apply(lambda x: len(x.split())))

    print('Calculating BLEU...')
    _eval[_k]['bleu'] = _bleu.compute(
        predictions=_predictions,
        references=_references
    )

    print('Calculating ROUGE...')
    _eval[_k]['rouge'] = _rouge.compute(
        predictions=_predictions,
        references=_references
    )

    print('Calculating METEOR...')
    _eval[_k]['meteor'] = _meteor.compute(
        predictions=_predictions,
        references=_references
    )

    print('Calculating BERTScore...')
    _eval[_k]['bertscore'] = _bertscore.compute(
        predictions=_predictions,
        references=_references,
        lang="en"
    )

    print('Calculating FrugalScore...')
    _eval[_k]['frugalscore'] = _frugalscore.compute(
        predictions=_predictions,
        references=_references,
        batch_size=16,
        max_length=64,
        device="gpu"
    )

    print('Calculating Google BLEU...')
    _eval[_k]['google_bleu'] = _google_bleu.compute(
        predictions=_predictions,
        references=_references
    )

    print('Calculating BLEURT...')
    _eval[_k]['bleurt'] = _bleurt.compute(
        predictions=_predictions,
        references=_references
    )

In [9]:
_eval

defaultdict(collections.defaultdict, {})

In [10]:
_df_result = pd.DataFrame.from_dict(_eval)
_df_result

""


In [11]:
_df_result = _df_result.T
_df_result

""


In [12]:
for _k in ['precision', 'recall', 'f1']:
    _df_result[f'bertscore_{_k}_mean'] = _df_result['bertscore'].apply(lambda _x: np.mean(_x[_k]))

_df_result[f'bertscore_{_k}_std'] = _df_result['bertscore'].apply(lambda _x: np.std(_x[_k]))

_df_result[f'frugalscore_{_k}_mean'] = _df_result['frugalscore'].apply(lambda _x: np.mean(_x['scores']))
_df_result[f'frugalscore_{_k}_std'] = _df_result['frugalscore'].apply(lambda _x: np.std(_x['scores']))

_df_result[f'bleurt_{_k}_mean'] = _df_result['bleurt'].apply(lambda _x: np.mean(_x['scores']))
_df_result[f'bleurt_{_k}_std'] = _df_result['bleurt'].apply(lambda _x: np.std(_x['scores']))

_sorted_columns = sorted(_df_result.columns.tolist())
_df_result['Model'] = _df_result.index
_df_result = _df_result[['Model'] + _sorted_columns]
_df_result

KeyError: 'bertscore'

In [ ]:
_combine_df = pd.concat([_df_existing_evals, _df_result], axis=0).reset_index(drop=True)
_combine_df

In [25]:
_combine_df.to_csv('llama2_eval.csv')

In [26]:
_df_no_arrays = _combine_df.drop(columns=['bleu', 'bertscore', 'frugalscore', 'bleurt', ])

In [27]:
_df_no_arrays

,Model,bertscore_f1_mean,bertscore_f1_std,bertscore_precision_mean,bertscore_recall_mean,bleurt_f1_mean,bleurt_f1_std,frugalscore_f1_mean,frugalscore_f1_std,google_bleu,meteor,predicted_utterances_count,predicted_words_count_mean,predicted_words_count_std,reference_words_count_mean,reference_words_count_std,rouge
0,finetuning/miti_alexander_street/meta-llama-Ll...,0.857203,0.043770,0.858164,0.857271,-0.963470,0.640698,0.006566,0.276434,{'google_bleu': 0.058391085206639076},{'meteor': 0.2036675683351478},6652,29.507817,33.912126,25.904991,29.444664,"{'rouge1': 0.2286281584299184, 'rouge2': 0.076..."
1,finetuning/alexander_street_small/meta-llama-L...,0.846896,0.029539,0.853084,0.842088,-1.145407,0.485106,-0.098164,0.157977,{'google_bleu': 0.03455417966787487},{'meteor': 0.1255303343638163},6652,24.511876,35.467151,25.904991,29.444664,"{'rouge1': 0.14487632150894864, 'rouge2': 0.02..."
2,finetuning/alexander_street_large/meta-llama-L...,0.848894,0.031134,0.858908,0.840558,-1.122640,0.513625,-0.097693,0.169693,{'google_bleu': 0.03292305001123111},{'meteor': 0.11923888191444706},6652,22.558178,33.688205,25.904991,29.444664,"{'rouge1': 0.1385491822036144, 'rouge2': 0.020..."
3,meta-llama/Llama-2-7b-chat-hf,0.810307,0.022486,0.781226,0.842561,-1.317601,0.441668,-0.094603,0.121680,{'google_bleu': 0.019129019671542388},{'meteor': 0.1631574496909516},6652,173.048256,33.08907,25.904991,29.444664,"{'rouge1': 0.10993872775780206, 'rouge2': 0.01..."
4,finetuning/alexander_street_large/meta-llama-L...,0.846356,0.028436,0.853447,0.840669,-1.172437,0.484274,-0.104471,0.151046,{'google_bleu': 0.03286954239250308},{'meteor': 0.11967210638136731},6652,23.107637,32.997601,25.904991,29.444664,"{'rouge1': 0.13922652976032834, 'rouge2': 0.02..."
5,finetuning/alexander_street_small/meta-llama-L...,0.846750,0.030293,0.852244,0.842509,-1.148234,0.494833,-0.093074,0.160853,{'google_bleu': 0.03529539773505573},{'meteor': 0.12678145049351427},6652,24.351022,33.650652,25.904991,29.444664,"{'rouge1': 0.1467090228221501, 'rouge2': 0.023..."
6,finetuning/miti_alexander_street/meta-llama-Ll...,0.861267,0.047765,0.864269,0.859231,-0.898174,0.691943,0.031425,0.310489,{'google_bleu': 0.07078532481310602},{'meteor': 0.21671404789969428},6652,24.929946,27.325215,25.904991,29.444664,"{'rouge1': 0.249585850179374, 'rouge2': 0.0966..."
7,finetuning/miti/meta-llama-Llama-2-13b-chat-hf-6,0.862704,0.050348,0.865171,0.861135,-0.858924,0.718365,0.050345,0.323301,{'google_bleu': 0.06486506624350302},{'meteor': 0.22925758259801637},6652,25.274354,28.995102,25.904991,29.444664,"{'rouge1': 0.2642435855610533, 'rouge2': 0.105..."
8,finetuning/miti/meta-llama-Llama-2-7b-chat-hf-6,0.858311,0.045633,0.858091,0.859440,-0.911828,0.670952,0.025398,0.292531,{'google_bleu': 0.05766905689656642},{'meteor': 0.2225239151728812},6652,33.305322,36.569563,25.904991,29.444664,"{'rouge1': 0.24632216278547844, 'rouge2': 0.08..."
9,meta-llama/Llama-2-13b-chat-hf,0.804415,0.030133,0.774571,0.837792,-1.331038,0.416551,-0.106791,0.113936,{'google_bleu': 0.020428763345130177},{'meteor': 0.16195078121252887},6652,166.406194,43.370973,25.904991,29.444664,"{'rouge1': 0.1126175778580799, 'rouge2': 0.020..."


In [29]:
_df_no_arrays.to_csv('llama2_eval_no_arrays.csv', index=False)